In [1]:
%pip install -r ../requirements.txt
# %pip install kaggle
# !kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews -p ../Dataset/

Note: you may need to restart the kernel to use updated packages.Collecting pandas (from -r ../requirements.txt (line 1))
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     --------------------------------- ------ 51.2/61.0 kB 2.6 MB/s eta 0:00:01
     ---------------------------------------- 61.0/61.0 kB 1.6 MB/s eta 0:00:00
  Using cached bs4-0.0.2-py2.py3-none-any.whl.metadata (411 bytes)
  Using cached langdetect-1.0.9.tar.gz (981 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/134.8 kB ? eta -:--:--
     -----------------

In [4]:
import pandas as pd
import numpy as np
import re
import string
import zipfile

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest, chi2
from urllib.parse import urlsplit
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

from bs4 import BeautifulSoup
from langdetect import detect
from urllib.parse import urlsplit

from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mahesh\AppData\Roaming\nltk_data...
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mahesh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mahesh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Mahesh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [ ]:
# Specify the path to the ZIP file
zip_file_path = r"../Datasets/imdb-dataset-of-50k-movie-reviews.zip"

# Open the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Find the first file with a .csv extension (assuming it's the one you want)
    csv_file = [name for name in zip_ref.namelist() if name.endswith('.csv')][0]
    
    # Read the CSV file directly from the ZIP archive into a DataFrame
    df = pd.read_csv(zip_ref.open(csv_file))

df.head()

In [ ]:
df_subset = df.sample(n=5000,random_state=42).reset_index(drop=True)
df_subset.head()

In [ ]:
df_subset['sentiment'].value_counts()

In [ ]:
#lowercase
df_subset["review"]=df_subset["review"].apply(lambda x:x.lower())

#Remove punctuations
df_subset['review'] = df_subset['review'].str.replace('[{}]'.format(string.punctuation), ' ')

# Remove numbers from the 'reviews' column
df_subset['review'] = df_subset['review'].str.replace(r'\d+', '')

# Stopwords

In [ ]:
# Function to remove stopwords from a text
def remove_stopwords(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    return ' '.join(filtered_tokens)

# Apply the remove_stopwords function to the 'review' column
df_subset['review'] = df_subset['review'].apply(remove_stopwords)

# Url

In [ ]:
def remove_urls(text):
    # Define a regular expression pattern to match URLs
    url_pattern = re.compile(r'https?://\S+|www\.\S+')

    # Find all matches in the text
    urls = re.findall(url_pattern, text)

    # Remove URLs from the text
    text_without_urls = re.sub(url_pattern, '', text)

    return text_without_urls

# Example usage
df_subset['review'] = df_subset['review'].apply(remove_urls)

# HTML

In [ ]:
def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

df_subset["review"] = df_subset["review"].apply(remove_html_tags)

# Non alphanumeric

In [ ]:
def clean_text(text):
    # Remove non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text

df_subset['review'] = df_subset['review'].apply(clean_text)

# Extra spaces

In [ ]:
def remove_extra_whitespaces(text):
    # Use regular expression to replace multiple whitespaces with a single space
    return re.sub(r'\s+', ' ', text).strip()

df_subset['review'] = df_subset['review'].apply(remove_extra_whitespaces)

# Filter non-English comments

In [ ]:
def filter_non_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

# Create a boolean mask for non-English reviews
mask = df_subset['review'].apply(filter_non_english)

# Create a new DataFrame containing only English reviews
df_subset = df_subset[mask]

# Lemma

In [ ]:
# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to get the part of speech for WordNet lemmatizer
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default to noun if the part of speech is not found

# Function to lemmatize a text
def lemmatize_text(text):
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    lemmatized_tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word, pos in pos_tags]
    return ' '.join(lemmatized_tokens)

# Apply lemmatization to the 'text' column
df_subset['review'] = df_subset['review'].apply(lemmatize_text)

In [ ]:
df_subset.head()

# LabelEncoding

In [ ]:
label = LabelEncoder()
df_subset['sentiment'] = label.fit_transform(df_subset['sentiment'])

In [ ]:
df_subset.head()

In [ ]:
# df_subset.to_csv('../Datasets/IMDB_Preprocessed.csv',index=False)

# Feature Extraction

In [ ]:
# Initialize the TF-IDF vectorizer
vectorizer = TfidfVectorizer()
totalFeatures = vectorizer.fit_transform(df_subset['review'])
column_names = vectorizer.get_feature_names_out()
totalFeatures.shape

In [ ]:
final_features = pd.DataFrame(totalFeatures.toarray(), columns=column_names)
final_features.head()

In [ ]:
final_features.to_csv('../Datasets/TFIDF_Features.csv', index=False)

# Feature Selection

# Classification